### This Code secured **first** position in the conducted during **Machine Learning Developer Summit(MLDS 2021)** held between 11-13th feb.
 
The hackathon was conducted by - [LEAPS](https://leapsapp.analyttica.com/home)
  
MLDS summit link - [MLDS link](https://virtualmlds.analyticsindiasummit.com/community/#/home)
  
hackathon link - [hackathon link](https://leapsapp.analyttica.com/hackathons/MLDS) 

## EDA

* Candidates who have studied hsc and ssc in central board have a higher chance of getting placed as compared to students who have studied hsc and ssc in stateboard.
* The proportion of train-test split done in the provided datasets - 70:30.
* In the training dataset, 27.3% of students were placed.
* Out of the placed population, 3.1% was female42.9% of candidates had taken science as hsc_stream.
* 37.4% of males are being placed in the total population.
* 'Commerce and mgmt' is the most popular degree type chosen by students. 
* 'work_exp' has the highest numbers of missing records.

## Approach
* The problem statement was a binary classification problem, where 'status' was the binary output, with values 1 or 0. Status denoted whether a student is placed or not.
* basic analysis and eda were done to find out the datatypes, missing values, outliers, and the distribution of the data.
* 'cat_convertor' function maps the categorical column values into numeric form.
* 'removing_outliers' function removes outliers is pct columns by fixing typos (4.89 -> 48.9), (778.98 -> 77.898). It also uses InterQuartile Range (IQR) to make the remaining outliers as NaN.
* For filling up the missing values, I used IterativeImputer from Sklearn with extratreesclassifier as an estimator, extratreesclassifier showed improvement above gbdt which is the default estimator.
* For categorical columns with integer values, After imputing the data I run the round(int()) command to again convert the float value to the nearest integer value.
* Adding PCA columns with 95% variance distribution bumped the numbers. MinMaxScaler was used on the dataset before PCA.
* I applied a cross_val_score for xgb, lgbm, catboost base models. catboost gave me the best results. Hence I decided to move with catboost + hyperparameter tuning.
* Things that didn't work out for me - a)smoothing of pct columns. b)SMOTE c) I tried to combined features like percentage of candidates divided by the average percentage of candidates grouped by gender/stream/degree_type ad total sum of percentage, the difference between percentage, etc

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# reading train and test dataset
train = pd.read_csv('../input/college-to-corporate-data/final_train.csv', sep=';')
test = pd.read_csv('../input/college-to-corporate-data/final_test.csv', sep=';')
del train['id']
del test['id']
train.shape,test.shape

In [ ]:
#basic analysis of dataset
train.isnull().sum()

In [ ]:
train.info()

In [ ]:
# function maps categorical in numeric form.
# degree_type had 3 unique values, other categorical columns had two unique vales/ Therefore I decided to not use one-hot encoding.
def cat_convertor(df):
    cat_mapping_dict = {
        'gender':{'Female':0,'Male':1},
        'ssc_board':{'Central Board':1,'State Board':0},
        'hsc_board':{'Central Board':1,'State Board':0},
        'hsc_stream':{'Commerce':0,'Science':1, 'Arts':2},
        'degree_type':{'Commerce & Mgmt':0,'Other Degree':1,'Science & Tech':2},
        'work_exp':{'No':0, 'Yes':1},
        'stream':{'Finance':0, 'HR':1}
    }
    df['gender'] = df['gender'].map(cat_mapping_dict['gender'])
    df['ssc_board'] = df['ssc_board'].map(cat_mapping_dict['ssc_board'])
    df['hsc_board'] = df['hsc_board'].map(cat_mapping_dict['hsc_board'])
    df['hsc_stream'] = df['hsc_stream'].map(cat_mapping_dict['hsc_stream'])
    df['degree_type'] = df['degree_type'].map(cat_mapping_dict['degree_type'])
    df['work_exp'] = df['work_exp'].map(cat_mapping_dict['work_exp'])
    df['stream'] = df['stream'].map(cat_mapping_dict['stream'])
    return df

train = cat_convertor(train)
test = cat_convertor(test)

In [ ]:
# Scope of improvement - my additional features didnot add much signal to the final score.

# def feature_engineering(df):
#     df['school_pct'] = df['ssc_pct']+df['hsc_pct']
#     df['school_pct_avg'] = df['school_pct']/2.0
    
#     df['college_pct'] = df['degree_pct']+df['mba_pct']
#     df['mba_pct_avg'] = df['school_pct']/2.0
    
#     df['all_pct'] = df['ssc_pct']+df['hsc_pct'] + df['degree_pct']+df['mba_pct']
#     df['all_pct_avg'] = df['all_pct']/4.0

#     temp = df.groupby('gender')['mba_pct'].median().reset_index(name='mba_pct_by_gender')
#     df = df.merge(temp, how='left', on='gender')
#     df['mba_pct_by_gender'] = df['mba_pct']/df['mba_pct_by_gender']
    
#     temp = df.groupby('degree_type')['degree_pct'].median().reset_index(name='degree_pct_by_degree_type')
#     df = df.merge(temp, how='left', on='degree_type')
#     df['degree_pct_by_degree_type'] = df['degree_pct']/df['degree_pct_by_degree_type']
    
#     return df

In [ ]:
# functions tries to remove typos from percent columns
# no body who has scored less than 33% get a valid certificate.
# no one can score more than 100#
# It's a common mistake of writing [ab.cd] to [a.bcd] or [abc.d], so I tried to fix this issue here. 
#converts 4.89 -> 48.9
#converts 987.23 -> 98.723
#converts 1003.58 -> 3.0 (scope of improvement)


def removing_outliers(series):
    IQR= series.quantile(.75) - series.quantile(.25)
    lower_boundary = series.quantile(.25) - (IQR*1.5) #36
    upper_boundary = series.quantile(.75) + (IQR*1.5) #100
    series = series.apply(lambda x: x*10 if x<10 else x)
    series = series.apply(lambda x: x*.1 if (x>100 and x<1000) else x)
    series = series.apply(lambda x: float(str(int(x))[2:]) if x>1000 else x)
    series = series.apply(lambda x: np.nan if x<lower_boundary else x)
    series = series.apply(lambda x: np.nan if x>upper_boundary else x)
    return series

train['ssc_pct'] = removing_outliers(train['ssc_pct'])
train['hsc_pct'] = removing_outliers(train['hsc_pct'])
train['degree_pct'] = removing_outliers(train['degree_pct'])
train['emptest_pct'] = removing_outliers(train['emptest_pct'])
train['mba_pct'] = removing_outliers(train['mba_pct'])

test['ssc_pct'] = removing_outliers(test['ssc_pct'])
test['hsc_pct'] = removing_outliers(test['hsc_pct'])
test['degree_pct'] = removing_outliers(test['degree_pct'])
test['emptest_pct'] = removing_outliers(test['emptest_pct'])
test['mba_pct'] = removing_outliers(test['mba_pct'])

In [ ]:
train.head()

In [ ]:
X = train.drop('status', axis=1)
y = train['status']
X.columns

In [ ]:
#using iterative imputer to fill up the missing values.
# using extratreesclassifier (it is fast compared to other trees methods) instead of gbdt(by default) gave me improvement.

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
imp_mean = IterativeImputer(estimator=ExtraTreesRegressor(n_estimators=10, random_state=0), random_state=0)
X_imputed = imp_mean.fit_transform(X)
test = imp_mean.fit_transform(test)

In [ ]:
#converting the categorical values to the nearest interger values
X = pd.DataFrame(X_imputed, columns=X.columns)
X['gender'] = X['gender'].apply(lambda x: int(round(x)))
X['ssc_board'] = X['ssc_board'].apply(lambda x: int(round(x)))
X['hsc_board'] = X['hsc_board'].apply(lambda x: int(round(x)))
X['hsc_stream'] = X['hsc_stream'].apply(lambda x: int(round(x)))
X['degree_type'] = X['degree_type'].apply(lambda x: int(round(x)))
X['work_exp'] = X['work_exp'].apply(lambda x: int(round(x)))
X['stream'] = X['stream'].apply(lambda x: int(round(x)))
X.head()

In [ ]:
test = pd.DataFrame(test, columns=X.columns)
test['gender'] = test['gender'].apply(lambda x: int(round(x)))
test['ssc_board'] = test['ssc_board'].apply(lambda x: int(round(x)))
test['hsc_board'] = test['hsc_board'].apply(lambda x: int(round(x)))
test['hsc_stream'] = test['hsc_stream'].apply(lambda x: int(round(x)))
test['degree_type'] = test['degree_type'].apply(lambda x: int(round(x)))
test['work_exp'] = test['work_exp'].apply(lambda x: int(round(x)))
test['stream'] = test['stream'].apply(lambda x: int(round(x)))
test.head()

In [ ]:
#pca gave me a minimal improvement.
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaledX = scaler.fit_transform(X)
scaledTest = scaler.transform(test)

pca = PCA(n_components=0.90,random_state=0)
pca.fit(scaledX)

pca_X = pca.transform(scaledX)
pca_Test = pca.transform(scaledTest)
print('pca_X.shape ',pca_X.shape)
print('pca_Test.shape', pca_Test.shape)

print('X.shape ',X.shape)
print('Test.shape', test.shape)

pca_X = pd.DataFrame(pca_X)
pca_Test = pd.DataFrame(pca_Test)

pca_X.columns = ['pca_'+str(col) for col in pca_X]
pca_Test.columns = ['pca_'+str(col) for col in pca_Test]

X = pd.concat([X,pca_X],axis=1)
test = pd.concat([test,pca_Test],axis=1)
print('combined X shape',X.shape)
print('combined test shape',test.shape)

In [ ]:
X.head()

In [ ]:
# didnot show any imrpovement

# X['ssc_pct'] = np.log1p(X['ssc_pct'])
# X['hsc_pct'] = np.log1p(X['hsc_pct'])
# X['degree_pct'] = np.log1p(X['degree_pct'])
# X['emptest_pct'] = np.log1p(X['emptest_pct'])
# X['mba_pct'] = np.log1p(X['mba_pct'])

# test['ssc_pct'] = np.log1p(test['ssc_pct'])
# test['hsc_pct'] = np.log1p(test['hsc_pct'])
# test['degree_pct'] = np.log1p(test['degree_pct'])
# test['emptest_pct'] = np.log1p(test['emptest_pct'])
# test['mba_pct'] = np.log1p(test['mba_pct'])

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=2020,test_size=0.20,shuffle=True,stratify=y)

In [ ]:
# xgb cross val

import xgboost as xgb
model_xgb = xgb.XGBClassifier(colsample_bylevel= 1, learning_rate= 0.1,max_depth=10, n_estimators= 1000, verbosity = 0)
result = cross_val_score(estimator=model_xgb,X=X_train,y=y_train,cv=10)
print(result)
print(result.mean())

# NOTE:
# The output of XGB before adding pca features were - 
# xgb
# 0.8912709022526982
# 0.8956617202207225
# 0.8901668383506782
# 0.8819781852691254
# 0.8926499994374159
# Average:  0.8903455291061281

In [ ]:
# lgbm cross val
import lightgbm as lgb
model_lgb = lgb.LGBMClassifier(learning_rate=0.2, n_estimators= 1000, verbosity = 0)
result = cross_val_score(estimator=model_lgb,X=X_train,y=y_train,cv=10)
print(result)
print(result.mean())

In [ ]:
# catboost cross val
from catboost import CatBoostClassifier
modelcat = CatBoostClassifier(random_state=42, max_depth=6, n_estimators=1000, verbose=0,learning_rate=.1)
result = cross_val_score(estimator=modelcat,X=X_train,y=y_train,cv=10)
print(result)
print(result.mean())

#### Seeing the above corss-val score, I decided to do feature engineering and hyperparameter tuning with only catboost in focus

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [ ]:
# SMOTE NOT WORKING
# from imblearn.over_sampling import SMOTE
# from collections import Counter

from catboost import CatBoostClassifier

final_preds = []
sfold, scores = StratifiedKFold(n_splits=5, shuffle=True, random_state=9), list()
for train_idx, test_idx in sfold.split(X, y):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    # num_class0, num_class1 = Counter(y_train)[0], Counter(y_train)[1]
    # sm = SMOTE(random_state=27, sampling_strategy={0: int(1.23*num_class0), 1: int(3.27*num_class1)})
    # x_train, y_train = sm.fit_resample(x_train, y_train)
    modelcat = CatBoostClassifier(random_state=0, max_depth=10, n_estimators=1000, verbose=500,learning_rate=.01)
    modelcat.fit(x_train, y_train)
    preds = modelcat.predict(x_test)
    score = f1_score(y_test, preds, average="weighted")
    scores.append(score)
    test_pred = modelcat.predict(test)
    final_preds.append(test_pred)
#         print(score)
    print("Average:----> ", sum(scores)/len(scores))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def feature_importance(model, X_train):
    fI = model.feature_importances_
    print(fI)
    names = X_train.columns.values
    ticks = [i for i in range(len(names))]
    plt.bar(ticks, fI)
    plt.xticks(ticks, names,rotation = 90)
    plt.show()

feature_importance(modelcat, X)

In [ ]:
# output is based on majority voting, output is what 3 or more models of catboost decide.
final_preds_df = pd.DataFrame(final_preds)
final_preds_ = final_preds_df.T
final_preds_['voting_output'] = final_preds_.sum(axis=1)
final_preds_['combined_output'] = final_preds_['voting_output'].apply(lambda x: 1 if x>2 else 0)
final_preds_.head()

In [ ]:
test_submission = pd.read_csv('../input/college-to-corporate-data/final_test.csv', sep=';')
test_submission['status'] = final_preds_['combined_output']
test_submission = test_submission[['id','status']]
test_submission.head(10)

### If you have read my notebook till here then please **UPVOTE** and **add comments**.
### If you have any doubts/suggestions then please write them below in the comments section. I'll try to answer them.